# Example of Speech to Text

This is an example of speech to text modules and using whisper to do transcriptions.

In this project we not only transcribe, but also summarize the transcriptions meeting summaries, indeintify speakers etc.

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
%%writefile "../whispermod.py"
from mangorest.mango import webapi 
import colabexts
from colabexts import jcommon
from pytube import YouTube
import whisper,  os, datetime, librosa, io, soundfile, sys, hashlib, torch
import threading
import numpy as np

model  = None
device = "cpu"
#-----------------------------models-----------------------------------------------------------------
device = "cpu"
if (torch.cuda.is_available() ):
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
#-----------------------------------------------------------------------------------
mutex  = threading.Lock()
def getmodel():
    global model, mutex
    
    if model is None:
        mutex.acquire()
        if model is None:
            model = whisper.load_model("base", device=device)
        mutex.release()   
    return model
#-----------------------------------------------------------------------------------
def transcribe_file(file ="/Users/snarayan/Desktop/data/audio/index.mp4", **kwargs):
    result = getmodel().transcribe(file)
    return result
#-----------------------------------------------------------------------------------
def is_video_file(filename):
    video_extensions = [".mp4", ".avi", ".mov", ".mkv", ".wmv", ".flv"] 
    return os.path.splitext(filename)[1].lower() in video_extensions
#-----------------------------------------------------------------------------------
def is_audio_file(filename):
    audio_extensions = [".mp3", ".wav", ".flac", ".ogg", ".aac", ".m4a"] 
    return os.path.splitext(filename)[1].lower() in audio_extensions
# ------------------------------------------------------------------------------
def transcribe(fn, offset=0, duration=60*60, detectSpeakers=1, **kwargs):
    if (type(fn) == str):
        data, sample_rate = librosa.load(fn, offset=offset, duration=duration, mono=True, sr=16000)
    else:
        content = io.BytesIO(fn)
        data, sample_rate = librosa.load(content,sr=16000, mono=True, offset=offset, duration=duration)

    ret = getmodel().transcribe(data)
    for s in ret.get('segments', []):
        s.pop('tokens')
        if ( detectSpeakers):
            speaker = who(data, s.get('start'), s.get('end'), **kwargs)
            s['speakers'] = [speaker]

    return ret

def transcribeWS(request, **kwargs):
    for f in request.FILES.getlist('file'):
        content = f.read()

    ret = transcribe(content, **kwargs)    
    print(f"Transcribed: {ret}")
    return ret

# ------------------------------------------------------------------------------
@webapi("/scribe/convert2wav/")
def convert2wav(request, **kwargs):
    for f in request.FILES.getlist('file'):
        content = f.read()

    content = io.BytesIO(content)
    data, sample_rate = librosa.load(content,sr=16000, mono=True, offset=offset, duration=duration)

    #offset, duration, file = 0, None, "/tmp/test1.wav"
    #data, sample_rate = librosa.load(file, offset=offset, duration=duration, mono=True, sr=16000)
    ret = io.BytesIO()
    soundfile.write("/tmp/_tmp.wav", data, samplerate=16000)
    with open("/tmp/_tmp.wav", "rb") as f:
        ret.write(f.read())
    return ret
#--------------------------------------------------------------------------------------------------------    
test_url = "https://www.youtube.com/watch?v=DuSDVj9a4WM&list=PLEpvS3HCVQ5_ZlyF1_i-WSwBzLoDLxoc9"

@webapi("/scribe/transcribe_youtube/")
def transcribe_youtube( url = test_url , force_download=False, force_transribe=False, **kwargs):    
    h = hashlib.md5(url.encode())
    file = "/tmp/" + str(h.hexdigest()) + ".mp4"
    
    if (force_download or not os.path.exists(file)):  
        file = YouTube(url).streams.filter(only_audio=True).first().download(filename=file)

    print( f"File: {file}")
    if (force_transribe or not os.path.exists(file +".txt")):  
        print( f"Calling transcription: {file}.txt")
        tr = getmodel().transcribe(file)
        ret = splitIntoParas(tr)
        with open(file +".txt", "w") as f:
            f.write(ret)
        with open(file +".json", "w") as f:
            f.write(str(tr))
            
        transcription = ret
    else:
        with open(file +".txt", "r") as f:
            transcription = f.read()
        
    return transcription;

#--------------------------------------------------------------------------------------------------------    
@webapi("/scribe/transcribe_media/")
def transcribe_me
( request, url=None, force_reload="", **kwargs):    
    print(f"{force_reload} <==========")
    ret  = [] 
    # STEP 1. extract from file
    for f in request.FILES.getlist('file'):
        content = f.read()
        filename = f"/tmp/{str(f)}"
        
        if ( not (is_video_file(filename) or is_audio_file(filename) ) ):
            ret.append (dict(file=filename, trans=f"{filename}: - not a media file", segs="") )
            continue;
        
        filename_trans = f"{filename}.trans"
        filename_segs  = f"{filename}.segs"
        
        if (not force_reload and os.path.exists(filename)):
            if (os.path.exists(filename_trans) and os.path.exists(filename_segs) ):
                with open(filename_trans) as f:
                    trans = f.read()
                with open(filename_segs) as f:
                    segs = f.read()
                    
                ret.append (dict(file=filename, trans= trans, segs=segs))
            else:
                msg = f"{filename}: file exists! have you initiated transcribe! click force"
                ret.append (dict(file=filename, trans=msg, segs="??") )
        else:
            with open(filename, "wb") as f:
                f.write(content)
            results = transcribe_file(filename)
            txt =""
            for t in results['segments']:
                o=f"{t['start']:-7.2f} - {t['end']:7.2f} : {t['text']}"
                txt += o + "\n"
            ret.append (dict(file=filename, trans= results['text'], segs=txt))
            
            with open(filename_trans, "w") as f:
                f.write(results['text'])
            with open(filename_segs, "w") as f:
                f.write(txt)
    
    # STEP 2: transcribe youtube URL when it works
    
    return ret;                

#-----------------------------------------------------------------------------------    
def process(sysargs):
    print("Parsing and processing")
    
    if (sysargs.url.strip()):
        print( f"Transcribing {sysargs.url}")
        transcribe_youtube(sysargs.url.strip())
    
#-----------------------------------------------------------------------------------
sysargs=None
def addargs():
    global sysargs
    p = argparse.ArgumentParser(f"{os.path.basename(sys.argv[0])}:")
    p.add_argument('-u', '--url', type=str, default="", help="Youtube URL")
    try:
        sysargs, unknown=p.parse_known_args(sys.argv[1:])
    except argparse.ArgumentError as exc:
        print(exc.message )
        
    if (unknown):
        print("Unknown options: ", unknown)
        #p.print_help()
    return sysargs    
#-----------------------------------------------------------------------------------
if __name__ == '__main__':
    if (not colabexts.jcommon.inJupyter()):
        t1 = datetime.datetime.now()
        sysargs = addargs()
        ret = process(sysargs)
        t2 = datetime.datetime.now()
        print(f"#All Done in {str(t2-t1)} ***")
    else:
        pass


Overwriting ../whispermod.py


## OLD

In [ ]:
%%writefile "../whispermod.py"
from mangorest.mango import webapi 
import colabexts
from colabexts import jcommon
from pytube import YouTube
import whisper,  os, datetime, librosa, io, soundfile, sys, hashlib
import threading
import numpy as np

#-----------------------------------------------------------------------------------
def splitIntoParas(tr, nLinesPerPara=4):
    n= nLinesPerPara
    l=tr.get('segments', [])
    ret = ""
    for i,j in enumerate(l[::n]):
        a, b = i*n, i*n + n
        o = "".join([j['text'] for j in l[a:b]])
        ret += o.strip() + "\n\n";
        #print(f'{a}-{b} {o} \n')
        
    return ret
#-----------------------------------------------------------------------------------
def who(data, start=0, end=None, **kwargs):
    try:
        import scribe.notebooks.speaker as speaker
        top3, tops = speaker.whoisInAudio1(data, top=3, start=start, end=end, **kwargs  )
        ret =  { 
            'top3'    : ";".join(top3[::-1]) ,
            'topScore': ""+str(tops)
        }
    except Exception as e:
        print("ERROR: " , e)
        ret = {'top3': ";;", 'topScore': 0}

    return ret

Overwriting ../whispermod.py


# Test

In [23]:
def get_video_id_from_url(video_url):
    """
    Examples:
    - http://youtu.be/SA2iWivDJiE
    - http://www.youtube.com/watch?v=_oPAwA_Udwc&feature=feedu
    - http://www.youtube.com/embed/SA2iWivDJiE
    - http://www.youtube.com/v/SA2iWivDJiE?version=3&amp;hl=en_US
    """
    import urllib.parse
    url = urllib.parse.urlparse(video_url)
    if url.hostname == 'youtu.be':
        return url.path[1:]
    if url.hostname in ('www.youtube.com', 'youtube.com'):
        if url.path == '/watch':
            p = urllib.parse.parse_qs(url.query)
            return p['v'][0]
        if url.path[:7] == '/embed/':
            return url.path.split('/')[2]
        if url.path[:3] == '/v/':
            return url.path.split('/')[2]

    return video_url

get_video_id_from_url( "http://www.youtube.com/watch?v=_oPAwA_Udwc&feature=feedu")
test_url = "https://www.youtube.com/watch?v=DuSDVj9a4WM&list=PLEpvS3HCVQ5_ZlyF1_i-WSwBzLoDLxoc9"
get_video_id_from_url(test_url)

'DuSDVj9a4WM'

In [1]:
import whisper

m=whisper.load_model("base")
m.transcribe("/tmp/test.wav")

/Users/e346104/venv/py312/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device

FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'

In [6]:
!ls /tmp/test.wav

/tmp/test.wav
